# Two Fluid Quantum Cosmological Model

In [5]:
# Importing the relevant libraries

import sys
import math
import numpy as np                                                # imports the Numpy Library

from tqdm import tqdm

import matplotlib.pyplot as plt

from numcosmo_py import Nc, Ncm                                   # imports the NumCosmo library
from numcosmo_py.plotting.tools import set_rc_params_article      # imports Numcosmo plotting tools

Ncm.cfg_init()                                                    # starts the NumCosmo library

# Introduction

In this notebook we shall analyze the predictions a two fluid quantum cosmological model where gravity is coupled though matter using the Wheeler-De Witt equation.

A single fluid with equation of state $p(\rho) = w\rho$ was considered in $\tt \text{arXiv:0610205}$ , using the framework of Bohmian mechanics. At background level, it lead to a non-singular scale factor. At perturbative level, the primordial spectrum of adiabatic perturbations was found to be approximately scale invariant, with

$${\cal P}_{\zeta}(k \gg 1) \approx A_{s}k^{n_{s} - 1}\, , $$
$$ n_{s} = 1 + \frac{12w}{1 + 3w}\, ,$$

for which the explicit value for dust $w \approx 0$ and radiation $w=1/3$ are given by

In [1]:
def nsw(w):                             # defines a function that calculates the single fluid spectral index ns
  return 1 + 12*w/(1+3*w)

print(r'w=0 => '+str(nsw(0)))           # dust w=0 spectral index
print(r'w=1/3 => '+str(nsw(1/3)))       # radiation w = 1/3 spectral index

w=0 => 1.0
w=1/3 => 3.0


The above results mean that ordinary matter cannot describe the approximately flat red spectrum with $n_{s}$. Therefore, a natural extension of said model is to consider the two fluid coupled to geometry.

Said model was analyzed at background level with great detail in $\tt \text{arXiv:0505109}$ , where it was found that the obtained scale factor is also non-singular. However, the perturbative analysis has not yet been performed due to theoretical difficulties.

Reference $\tt \text{arXiv:1510.06628}$ analyzes the cosmological perturbations formalism for $N$ fluid models. In the 2 fluid case, the obtained Hamiltonian that describes perturbations is given explicitly by

\begin{align}
    \delta {\cal H}^{(2,s)} & = \frac{1}{2m_{z}}P_{\zeta}^{2} + \frac{1}{2m_{S}}P_{Q}^{2} + \left({ \frac{ \bar{c}_{n} }{ \bar{c}_{S}\bar{c}_{m} } }^{2}\frac{1}{m_{z}m_{S}}\frac{1}{NH}\right)P_{\zeta}P_{Q} \\
    %
    & \, \, + \frac{1}{2}m_{z}\nu_{\zeta}^{2} z^{2} + \frac{1}{2}m_{S}\nu_{S}^{2}Q^{2} \, ,
\end{align}

where $\zeta$ and $Q$ denote adiabatic and entropy perturbations, respectively. We also introduced

\begin{align*}
    m_{z}  \equiv \frac{ a^{3}({ \bar{\rho} + \bar{p} }) }{N\bar{c}_{S}^{2}\bar{H}^{2} }\, , \hspace{2.4cm} & \hspace{-0.1cm} m_{S}  \equiv \frac{ 1 }{  N a^{3}\bar{c}_{m}^2\bar{\omega} } \, ,  \\
    %
    \nu_{z}^{2} \equiv \bar{c}_{S}^{2}F^{2}_{k} \, , \hspace{3.15cm} & \hspace{0.05cm} \nu_{S}^{2} \equiv {c}_{m}^{2}F^{2}_{k} \, ,  \\
    %
    \bar{c}_{S}^{2} \equiv \bar{c}_{1}^{2}\cos^{2}\phi+\bar{c}_{2}^{2}\sin^{2}\phi \, , \hspace{1cm} & \bar{c}_{m}^{2} \equiv \bar{c}_{2}^{2}\cos^{2}\phi+\bar{c}_{1}^{2}\sin^{2}\phi
    \, .
\end{align*}

where $\bar{\omega} \equiv (\rho_1 + p_1)(\rho_2 + p_2)/(\rho + p)$ and $\bar{c}_{n}^{2} \equiv \bar{c}_{1}^{2} - \bar{c}_{2}^{2}$. Here, for later convenience, we have also introduced the angular variable $\phi$ and the functions $F_{k}(t)$ by

\begin{equation}
 \cos^{2}\phi \equiv \frac{ \rho_1 + p_1 }{ \rho + p }\, ,
\end{equation}

\begin{equation}
 \sin^{2}\phi \equiv \frac{ \rho_2 + p_2 }{ \rho + p }\, ,
\end{equation}

\begin{equation}
 \hspace{0.8cm} F^{2}_{k} \equiv \left({\frac{ Nk }{ a }}\right)^{2}\, ,
\end{equation}

the angular variable $\phi$ is also associated to the dominant fluid, with $\phi=0$ denoting domination by the fluid $1$ and $\phi=\pi/2$ denoting domination by the fluid $2$.


By direct inspection of the perturbative Hamiltonian, one sees that the equations of motion for perturbations are coupled. While at classical level this could be solved by numerical methods, at quantum level this demands the use of special quantization techniques to define the associated vacuum state and extract predictions, as proposed by $\tt \text{arXiv:1510.06628}$.

Therefore in this notebook, we analyze explicitly the background and perturbative quantities, with our final aim to calculate the modes $\zeta_{k}, Q_{k}$ and their associated power spectrum.

# Background Quantities

In this section we study the quantities $m_{S}, m_{Q}, \phi, c^{2}_{S}, c^{2}_{m}$ that occur in the perturbative Hamiltonian. We also study the evolution of the eigenvalues $\nu_{1}, \nu_{2}$ that diagonalize the Hamiltonian tensor. Finally, we study the behavior of the coupling matrices $\gamma_{ij}, \tau_{ij}$, which define the dynamics of the relevant perturbative quantities.

To do so, we start by defining our cosmological parameters.

In [2]:
cosmo = Nc.HICosmoQGRW()                                    # defines a cosmological model, which is represented by a NumCosmo object. Then, the relevant cosmological parameters are added to this object

# these lines set the relevant cosmological parameters
cosmo.props.w = 1.0e-2                                      # dust/dark matter equation of state
cosmo.props.Omegar = 2.0 * (1.0e-8)                         # radiation abundance today
cosmo.props.Omegaw = 2.0 * (1.0 - 1.0e-8)                   # dust/dark matter abundance today
cosmo.props.xb = 1.0e30                                     # inverse scale factor x=1/a at the time of the bounce

NameError: name 'Nc' is not defined

To study the behavior of said quantities, we shall vary our cosmological parameters. However, before we dwell into such analysis, let's perform a consistency check by considering the dust only $\Omega_{r} = 0$ and radiation only $\Omega_{w} = 0$ cases. The complete model shall present regimes of domination by each fluid, and we shall also check if we indeed recover the $n_{s}$ values for the single fluid case.

## Consistency Check

## Evolution of ....

In [4]:
k = 1.0
min_alpha_c = -120.0
max_alpha_c = -1.0
min_alpha_scale = 1.0e-12
np_plot = 100

# Time arrays for the contraction and bounce phases

alpha_c = np.linspace(min_alpha_c, max_alpha_c, np_plot)
alpha_b_e = np.geomspace(min_alpha_scale, 2.0, np_plot)
alpha_b = np.concatenate((np.flip(-alpha_b_e), alpha_b_e))

In [ ]:
cosmo.eom_eval(-120,-1)
#m_s_c = np.array([cosmo.eom_eval(alpha, k).m_s for alpha in alpha_c])
#print(m_s_c)
#m_zeta_c = np.array([cosmo.eom_eval(alpha, k).m_zeta for alpha in alpha_c])

<NumCosmo.HIPertITwoFluidsEOM object at 0x7f3df17da3f0 (NcHIPertITwoFluidsEOM at 0x39c9560)>

In [ ]:
cosmo.eom_eval(-120,1).nu1
#cosmo.props.m_s

In [ ]:
# Computing background observables in the contraction phase

m_s_c = np.array([cosmo.eom_eval(alpha, k).m_s for alpha in alpha_c])
m_zeta_c = np.array([cosmo.eom_eval(alpha, k).m_zeta for alpha in alpha_c])
mnu2_s_c = np.array([cosmo.eom_eval(alpha, k).mnu2_s for alpha in alpha_c])
mnu2_zeta_c = np.array([cosmo.eom_eval(alpha, k).mnu2_zeta for alpha in alpha_c])
nu1_c = np.array([cosmo.eom_eval(alpha, k).nu1 for alpha in alpha_c])
nu2_c = np.array([cosmo.eom_eval(alpha, k).nu2 for alpha in alpha_c])
nu_s_c = np.sqrt(mnu2_s_c / m_s_c)
nu_zeta_c = np.sqrt(mnu2_zeta_c / m_zeta_c)
y_c = np.array([cosmo.eom_eval(alpha, k).y for alpha in alpha_c])
gamma11_c = np.array([cosmo.eom_eval(alpha, k).gammabar11 for alpha in alpha_c])
gamma22_c = np.array([cosmo.eom_eval(alpha, k).gammabar22 for alpha in alpha_c])
gamma12_c = np.array([cosmo.eom_eval(alpha, k).gammabar12 for alpha in alpha_c])
tau_c = np.array([cosmo.eom_eval(alpha, k).taubar for alpha in alpha_c])

# Computing background observables in the bounce phase

m_s_b = np.array([cosmo.eom_eval(alpha, k).m_s for alpha in alpha_b])
m_zeta_b = np.array([cosmo.eom_eval(alpha, k).m_zeta for alpha in alpha_b])
mnu2_s_b = np.array([cosmo.eom_eval(alpha, k).mnu2_s for alpha in alpha_b])
mnu2_zeta_b = np.array([cosmo.eom_eval(alpha, k).mnu2_zeta for alpha in alpha_b])
nu1_b = np.array([cosmo.eom_eval(alpha, k).nu1 for alpha in alpha_b])
nu2_b = np.array([cosmo.eom_eval(alpha, k).nu2 for alpha in alpha_b])
nu_s_b = np.sqrt(mnu2_s_b / m_s_b)
nu_zeta_b = np.sqrt(mnu2_zeta_b / m_zeta_b)
y_b = np.array([cosmo.eom_eval(alpha, k).y for alpha in alpha_b])
gamma11_b = np.array([cosmo.eom_eval(alpha, k).gammabar11 for alpha in alpha_b])
gamma22_b = np.array([cosmo.eom_eval(alpha, k).gammabar22 for alpha in alpha_b])
gamma12_b = np.array([cosmo.eom_eval(alpha, k).gammabar12 for alpha in alpha_b])
tau_b = np.array([cosmo.eom_eval(alpha, k).taubar for alpha in alpha_b])

cos2_phi_c = (nu1_c**2 * nu_zeta_c**2 - nu2_c**2 * nu_s_c**2) / (nu1_c**4 - nu2_c**4)
sin2_phi_c = (nu1_c**2 * nu_s_c**2 - nu2_c**2 * nu_zeta_c**2) / (nu1_c**4 - nu2_c**4)

cos2_phi_b = (nu1_b**2 * nu_zeta_b**2 - nu2_b**2 * nu_s_b**2) / (nu1_b**4 - nu2_b**4)
sin2_phi_b = (nu1_b**2 * nu_s_b**2 - nu2_b**2 * nu_zeta_b**2) / (nu1_b**4 - nu2_b**4)



In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(2,2,2)
ax3= fig.add_subplot(2,2,4)

ax1.plot(alpha_c, m_s_c, c='r', label=r'$m_s$')
ax1.plot(alpha_c, m_zeta_c, c='b', label=r'$m_\zeta$')
ax1.set_yscale('log')
ax1.set_xlabel(r'$\alpha$')

ax2.plot(alpha_b, m_s_b, c='r', label=r'$m_s$')
ax2.set_xscale('symlog', linthresh=min_alpha_scale)
ax2.set_yscale('log')
ax2.set_xlabel(r'$\alpha$')

ax3.plot(alpha_b, m_zeta_b, c='b', label=r'$m_\zeta$')
ax3.set_xscale('symlog', linthresh=min_alpha_scale)
ax3.set_yscale('log')
ax3.set_xlabel(r'$\alpha$')

ax1.legend()

pass

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(1,2,2)

ax1.plot(alpha_c, mnu2_s_c / m_s_c, c='r', label=r'$\nu_s^2$')
ax1.plot(alpha_c, mnu2_zeta_c / m_zeta_c, c='b', label=r'$\nu_\zeta^2$')
ax1.set_yscale('log')
ax1.set_xlabel(r'$\alpha$')
ax1.legend()

ax2.plot(alpha_b, mnu2_s_b / m_s_b, c='r', label=r'$\nu_s^2$')
ax2.plot(alpha_b, mnu2_zeta_b / m_zeta_b, c='b', label=r'$\nu_\zeta^2$')
ax2.set_xscale('symlog', linthresh=min_alpha_scale)
ax2.set_yscale('log')
ax2.set_xlabel(r'$\alpha$')

pass

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(1,2,2)

ax1.plot(alpha_c, y_c * np.sqrt(m_s_c * m_zeta_c), c='k', label=r'$\nu_s^2$')
ax1.set_yscale('symlog', linthresh=1.0e-10)
ax1.set_xlabel(r'$\alpha$')

ax2.plot(alpha_b, y_b * np.sqrt(m_s_b * m_zeta_b), c='k', label=r'$\nu_s^2$')
ax2.set_xscale('symlog', linthresh=min_alpha_scale)
ax2.set_yscale('symlog')
ax2.set_xlabel(r'$\alpha$')

ax1.legend()

pass

In [ ]:
set_rc_params_article(ncol=1)
fig = plt.figure()

ax1= fig.add_subplot(1,1,1)

ax1.plot(alpha_c, cos2_phi_c, label=r'$\cos^2(\phi)$')
ax1.plot(alpha_c, sin2_phi_c, label=r'$\sin^2(\phi)$')
ax1.set_xlabel(r'$\alpha$')
ax1.legend()
pass

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(1,2,2)

ax1.plot(alpha_c, nu1_c, label=r'$\nu_1$')
ax1.plot(alpha_c, gamma11_c, label=r'$\gamma_{11}$')
ax1.plot(alpha_c, gamma12_c, label=r'$\gamma_{12}$')
ax1.plot(alpha_c, tau_c, label=r'$\tau_{12}$')

ax1.set_yscale('symlog', linthresh=1.0e-6)
ax1.set_xlabel(r'$\alpha$')
ax1.legend()

ax2.plot(alpha_c, nu2_c, label=r'$\nu_2$')
ax2.plot(alpha_c, gamma22_c, label=r'$\gamma_{22}$')
ax2.plot(alpha_c, gamma12_c, label=r'$\gamma_{12}$')
ax2.plot(alpha_c, tau_c, label=r'$\tau_{12}$')

ax2.set_yscale('symlog', linthresh=1.0e-6)
ax2.set_xlabel(r'$\alpha$')
ax2.legend()

pass

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(1,2,2)

ax1.plot(alpha_b, nu1_b, label=r'$\nu_1$')
ax1.plot(alpha_b, gamma11_b, label=r'$\gamma_{11}$')
ax1.plot(alpha_b, gamma12_b, label=r'$\gamma_{12}$')
ax1.plot(alpha_b, tau_b, label=r'$\tau_{12}$')
ax1.set_xscale('symlog', linthresh=min_alpha_scale)
ax1.set_yscale('symlog', linthresh=1.0e-6)
ax1.set_xlabel(r'$\alpha$')

ax1.legend()

ax2.plot(alpha_b, nu2_b, label=r'$\nu_2$')
ax2.plot(alpha_b, gamma22_b, label=r'$\gamma_{22}$')
ax2.plot(alpha_b, gamma12_b, label=r'$\gamma_{12}$')
ax2.plot(alpha_b, tau_b, label=r'$\tau_{12}$')
ax2.set_xscale('symlog', linthresh=min_alpha_scale)
ax2.set_yscale('symlog', linthresh=1.0e-6)
ax2.set_xlabel(r'$\alpha$')
ax2.legend()

pass

# Perturbative Quantities

In [ ]:
def get_zeta(v):
    return v.get(Nc.HIPertITwoFluidsVars.ZETA_R) + 1.0j * v.get(Nc.HIPertITwoFluidsVars.ZETA_I)

def get_S(v):
    return v.get(Nc.HIPertITwoFluidsVars.S_R) + 1.0j * v.get(Nc.HIPertITwoFluidsVars.S_I)

def get_Pzeta(v):
    return v.get(Nc.HIPertITwoFluidsVars.PZETA_R) + 1.0j * v.get(Nc.HIPertITwoFluidsVars.PZETA_I)

def get_PS(v):
    return v.get(Nc.HIPertITwoFluidsVars.PS_R) + 1.0j * v.get(Nc.HIPertITwoFluidsVars.PS_I)


In [ ]:
def integrate_system(k):
    # Defining relative tolerance for integration
    prec = 1.0e-10
    # Ratio potential frequency to define cross time
    cross_size = 1.0e-9

    # New perturbations object
    pert1 = Nc.HIPertTwoFluids.new()
    pert2 = Nc.HIPertTwoFluids.new()
    # Setting reltol
    pert1.props.reltol = prec
    pert2.props.reltol = prec
    # Setting k
    pert1.set_mode_k(k)
    pert2.set_mode_k(k)

    # Choose an initial condition
    alpha_try = -cosmo.abs_alpha(1.0e-14 * k**2)

    # New vector to store initial conditions
    # 8 dimensional (Q_1, Q_2, P_1, P_2), real and imaginary parts
    ci1 = Ncm.Vector.new(8)
    ci2 = Ncm.Vector.new(8)

    alphai1 = pert1.get_cross_time(cosmo, Nc.HIPertTwoFluidsCross.MODE1MAIN, alpha_try, cross_size)
    alphai2 = pert2.get_cross_time(cosmo, Nc.HIPertTwoFluidsCross.MODE2MAIN, alpha_try, cross_size)

    # Compute initial conditions at alpha_try store at ci and use normalization factor
    # pi/4
    pert1.get_init_cond_zetaS(cosmo, alphai1, 1, 0.25 * math.pi, ci1)
    pert2.get_init_cond_zetaS(cosmo, alphai2, 2, 0.25 * math.pi, ci2)

    # Use the previously computed initial conditions to start the system at alpha_try
    pert1.set_init_cond(cosmo, alphai1, 30, False, ci1)
    pert2.set_init_cond(cosmo, alphai2, 30, False, ci2)
    # print(f"Setting initial conditions for zeta1 and S1 at {alphai1}")
    # print(f"Setting initial conditions for zeta2 and S2 at {alphai2}")

    if alphai2 > alphai1:
        pert1.evolve(cosmo, alphai2)
        ci1, _ = pert1.peek_state(cosmo)
    else:
        pert2.evolve(cosmo, alphai1)
        ci2, _ = pert2.peek_state(cosmo)

    alphai = max(alphai1, alphai2)

    # Create a array of times to integrate the system over
    alpha_evol = np.linspace(alphai, -1.0e-1, 1000)

    # Integrate the system by stepping through alpha_evol using .evolve
    zeta1_a = [get_zeta(ci1)]
    S1_a = [get_S(ci1)]
    Pzeta1_a = [get_Pzeta(ci1)]
    PS1_a = [get_PS(ci1)]

    zeta2_a = [get_zeta(ci2)]
    S2_a = [get_S(ci2)]
    Pzeta2_a = [get_Pzeta(ci2)]
    PS2_a = [get_PS(ci2)]

    for alpha in tqdm(alpha_evol[1:], desc="Time evolution", position=1, leave=False):
        pert1.evolve(cosmo, alpha)
        pert2.evolve(cosmo, alpha)
        v1, _alphac1 = pert1.peek_state(cosmo)
        v2, _alphac2 = pert2.peek_state(cosmo)

        zeta1_a.append(get_zeta(v1))
        S1_a.append(get_S(v1))
        Pzeta1_a.append(get_Pzeta(v1))
        PS1_a.append(get_PS(v1))

        zeta2_a.append(get_zeta(v2))
        S2_a.append(get_S(v2))
        Pzeta2_a.append(get_Pzeta(v2))
        PS2_a.append(get_PS(v2))

    zeta1 = np.array(zeta1_a)
    S1 = np.array(S1_a)
    Pzeta1 = np.array(Pzeta1_a)
    PS1 = np.array(PS1_a)

    zeta2 = np.array(zeta2_a)
    S2 = np.array(S2_a)
    Pzeta2 = np.array(Pzeta2_a)
    PS2 = np.array(PS2_a)

    return (alpha_evol, zeta1, S1, Pzeta1, PS1, zeta2, S2, Pzeta2, PS2)


In [ ]:
alpha_evol, zeta1, S1, Pzeta1, PS1, zeta2, S2, Pzeta2, PS2 = integrate_system(1.0)

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(2,1,1)
ax2= fig.add_subplot(2,1,2)

ax1.plot(alpha_evol, np.real(zeta1), label=r'$\mathrm{Re}(\zeta_1)$')
ax1.plot(alpha_evol, np.imag(zeta1), label=r'$\mathrm{Im}(\zeta_1)$')
ax1.plot(alpha_evol, np.real(zeta2), label=r'$\mathrm{Re}(\zeta_2)$')
ax1.plot(alpha_evol, np.imag(zeta2), label=r'$\mathrm{Im}(\zeta_2)$')
ax1.set_yscale('symlog')
ax1.set_xlabel(r'$\alpha$')
ax1.legend()

ax2.plot(alpha_evol, np.real(S1), label=r'$\mathrm{Re}(S_1)$')
ax2.plot(alpha_evol, np.imag(S1), label=r'$\mathrm{Im}(S_1)$')
ax2.plot(alpha_evol, np.real(S2), label=r'$\mathrm{Re}(S_2)$')
ax2.plot(alpha_evol, np.imag(S2), label=r'$\mathrm{Im}(S_2)$')
ax2.set_yscale('symlog')
ax2.set_xlabel(r'$\alpha$')
ax2.legend()


In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(2,1,1)
ax2= fig.add_subplot(2,1,2)

ax1.plot(alpha_evol, np.abs(zeta1)**2, label=r'$|\zeta_1|^2$')
ax1.plot(alpha_evol, np.abs(zeta2)**2, label=r'$|\zeta_2|^2$')
ax1.set_yscale('log')
ax1.set_xlabel(r'$\alpha$')
ax1.legend()

ax2.plot(alpha_evol, np.abs(S1)**2, label=r'$|S_1|^2$')
ax2.plot(alpha_evol, np.abs(S2)**2, label=r'$|S_2|^2$')
ax2.set_yscale('log')
ax2.set_xlabel(r'$\alpha$')
ax2.legend()

pass

In [ ]:
k_a = np.geomspace(1.0e-3, 1.0e3, 100)

PI_zeta1 = []
PI_zeta2 = []
PI_S1 = []
PI_S2 = []

for k in tqdm(k_a, desc= "Mode evolution", position=0):
    alpha_evol, zeta1, S1, Pzeta1, PS1, zeta2, S2, Pzeta2, PS2 = integrate_system(k)
    PI_zeta1.append(np.abs(zeta1[-1])**2)
    PI_zeta2.append(np.abs(zeta2[-1])**2)
    PI_S1.append(np.abs(S1[-1])**2)
    PI_S2.append(np.abs(S2[-1])**2)

In [ ]:
set_rc_params_article(ncol=2)
fig = plt.figure()

ax1= fig.add_subplot(2,1,1)
ax2= fig.add_subplot(2,1,2)

ax1.plot(k_a, k_a**3 * PI_zeta1, label=r'$\Pi_{\zeta_1}$')
ax1.plot(k_a, k_a**3 * PI_zeta2, label=r'$\Pi_{\zeta_2}$')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$k$')
ax1.legend()

ax2.plot(k_a, k_a**3 * PI_S1, label=r'$\Pi_{S_1}$')
ax2.plot(k_a, k_a**3 * PI_S2, label=r'$\Pi_{S_2}$')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'$k$')
ax2.legend()

pass

In [ ]:
#print(np.polyfit(np.log(k_a),np.log(k_a**3 * PI_zeta2),1)[0])

print(1.0 + 12.0 * cosmo.props.w / (1.0 + 3.0 *cosmo.props.w))


# Parte do Eduado

In [ ]:
def test_two_fluids_wkb_mode(mode: int = Nc.HIPertTwoFluidsCross.MODE1SUB, case_f: int = 1, nullr = 1.0, nulli = 1.0) -> None:
    """Compute WKB approximation for the two-fluids model."""

    #
    #  New homogeneous and isotropic cosmological model NcHICosmoQGRW
    #
    cosmo = Nc.HICosmoQGRW()

    w = 0.00001
    prec = 1.0e-6
    mode_k = 1000

    cosmo.props.w = w
    cosmo.props.Omegar = 2.0 * (1.0e-8)
    cosmo.props.Omegaw = 2.0 * (1.0 - 1.0e-8)
    cosmo.props.xb = 1.0e30

    pert = Nc.HIPertTwoFluids.new()

    pert.props.reltol = prec
    pert.set_mode_k(mode_k)

    cross_size = 1.0e-5
    alpha_try = -cosmo.abs_alpha(1.0e-12 * mode_k**2)
    # Chuta tempo incial e depois calcula ele aqui embaixo, dependendo de qual modo é o main

    # Pra descobrir se o alpha1 da outra funcao faz sentido, coloca auqi o caso que vc quiser e imprime o tempo calcualdo (ex: mode1main, da um alpha 1.0e-9. Por isso ele deve ter colocado na 2 funçao)
    if mode == Nc.HIPertTwoFluidsCross.MODE1MAIN:
        alphai = pert.get_cross_time(
            cosmo, Nc.HIPertTwoFluidsCross.MODE1MAIN, alpha_try, cross_size
        )
    elif mode == Nc.HIPertTwoFluidsCross.MODE1SUB:
        alphai = pert.get_cross_time(
            cosmo, Nc.HIPertTwoFluidsCross.MODE1SUB, alpha_try, cross_size
        )
    elif mode == Nc.HIPertTwoFluidsCross.MODE2MAIN:
        alphai = pert.get_cross_time(
            cosmo, Nc.HIPertTwoFluidsCross.MODE2MAIN, alpha_try, cross_size
        )
    elif mode == Nc.HIPertTwoFluidsCross.MODE2SUB:
        alphai = pert.get_cross_time(
            cosmo, Nc.HIPertTwoFluidsCross.MODE2SUB, alpha_try, cross_size
        )
    else:
        raise ValueError("Invalid mode")

    alphaf = +cosmo.abs_alpha(1.0e20)

    print(f"Mode k = mode_k: {mode_k}")

    pert.set_stiff_solver(True)

    alpha_a = []
    gammabar11_a = []
    gammabar22_a = []
    gammabar12_a = []
    taubar12_a = []
    nu1_a = []
    nu2_a = []

    for alpha in np.linspace(alphai, alphaf, 10000):
        eom = pert.eom(cosmo, alpha)
        alpha_a.append(alpha)

        gammabar11_a.append(math.fabs(eom.gammabar11))
        gammabar22_a.append(math.fabs(eom.gammabar22))
        gammabar12_a.append(math.fabs(eom.gammabar12))
        taubar12_a.append(math.fabs(eom.taubar))
        nu1_a.append(eom.nu1)
        nu2_a.append(eom.nu2)

    print(
        f"# Calculating mode 1, initial time {alphai}, redshift_alpha {cosmo.x_alpha(alphai):8.2e}]: "
    )

    ci = Ncm.Vector.new(8)

    pert.get_init_cond_zetaS(cosmo, alphai, case_f, 0.25 * math.pi, ci)
    pert.set_init_cond(cosmo, alphai, 3, False, ci)

    Ps_zeta1 = []
    Ps_S1 = []
    Ps_Pzeta1 = []
    Ps_PS1 = []

    Ps_zeta1.append(
        math.hypot(
            nullr * ci.get(Nc.HIPertITwoFluidsVars.ZETA_R),
            nulli * ci.get(Nc.HIPertITwoFluidsVars.ZETA_I),
        )
        ** 2
    )
    Ps_S1.append(
        math.hypot(
            nullr *ci.get(Nc.HIPertITwoFluidsVars.S_R),
            nulli *ci.get(Nc.HIPertITwoFluidsVars.S_I),
        )
        ** 2
    )
    Ps_Pzeta1.append(
        math.hypot(
            nullr *ci.get(Nc.HIPertITwoFluidsVars.PZETA_R),
            nulli *ci.get(Nc.HIPertITwoFluidsVars.PZETA_I),
        )
        ** 2
    )
    Ps_PS1.append(
        math.hypot(
           nullr * ci.get(Nc.HIPertITwoFluidsVars.PS_R),
           nulli * ci.get(Nc.HIPertITwoFluidsVars.PS_I),
        )
        ** 2
    )

    for alpha in tqdm(alpha_a[1:]):
        # for alpha in alpha_a[1:]:
        pert.evolve(cosmo, alpha)
        v, _alphac = pert.peek_state(cosmo)
        Ps_zeta1.append(
            math.hypot(
              nullr *  v.get(Nc.HIPertITwoFluidsVars.ZETA_R),
              nulli *  v.get(Nc.HIPertITwoFluidsVars.ZETA_I),
            )
            ** 2
        )
        Ps_S1.append(
            math.hypot(
               nullr * v.get(Nc.HIPertITwoFluidsVars.S_R),
              nulli *  v.get(Nc.HIPertITwoFluidsVars.S_I),
            )
            ** 2
        )
        Ps_Pzeta1.append(
            math.hypot(
              nullr *  v.get(Nc.HIPertITwoFluidsVars.PZETA_R),
              nulli *  v.get(Nc.HIPertITwoFluidsVars.PZETA_I),
            )
            ** 2
        )
        Ps_PS1.append(
           math.hypot(
              nullr *  v.get(Nc.HIPertITwoFluidsVars.PS_R),
             nulli *   v.get(Nc.HIPertITwoFluidsVars.PS_I),
            )
            ** 2
        )
    plt.plot(alpha_a, Ps_zeta1, label=r"$P_\zeta$")
    plt.plot(alpha_a, Ps_S1, label=r"$P_Q$")
    plt.plot(alpha_a, Ps_Pzeta1, label=r"$P_{P_\zeta}$")
    plt.plot(alpha_a, Ps_PS1, label=r"$P_{P_Q}$")
    plt.xlabel(r"$\alpha$")
    plt.ylabel(r"Mode")
    plt.grid()
    plt.legend(loc="upper left")
    # plt.xscale('log')
    plt.yscale("log")

    Delta_zeta1 = (
        mode_k**3 * Ps_zeta1.pop() / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
    )
    Delta_S1 = mode_k**3 * Ps_S1.pop() / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
    Delta_Pzeta1 = (
        mode_k**3 * Ps_Pzeta1.pop() / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
    )
    Delta_PS1 = (
        mode_k**3 * Ps_PS1.pop() / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
    )
    print(
        f"# Final values k = {mode_k: 20.15g} Ps_zeta{case_f} = {Delta_zeta1: 21.15e} "
        f"Ps_Pzeta{case_f} = {Delta_Pzeta1: 21.15e} Ps_S{case_f} = {Delta_S1: 21.15e} "
        f"Ps_PS{case_f} = {Delta_PS1: 21.15e}"
            )



    #plt.savefig('mode_I_p_R')
    #plt.clf()
    plt.show()

In [ ]:
def test_two_fluids_wkb_spec() -> None:
    """Compute WKB approximation for the two-fluids model spectrum."""

    #
    #  New homogeneous and isotropic cosmological model NcHICosmoQGRW
    #
    cosmo = Nc.HICosmoQGRW() # P1 = Cosmologia

    w = 1.0e-5 #P2 eq de estado da materia escura
    prec = 1.0e-7

    #P3 densidade de radiação e materia escura e energia escura (1 - resto)
    cosmo.props.w = w
    cosmo.props.Omegar = (1.0e-8) * 1.0
    cosmo.props.Omegaw = (1.0 - 1.0e-8) * 1.0
    cosmo.props.xb = 1.0e30 # P4 x do bounce

    pert = Nc.HIPertTwoFluids.new()

    pert.props.reltol = prec
    # pert.set_stiff_solver (True)

    # k = 1 => lambda = 5 Gpc
    # k = 10^5 => lambda = 50 kpc
    lnki = math.log(1.0e0)# P5 intervalo de momento de interesse
    lnkf = math.log(1.0e5)
    lnk_a = np.linspace(lnki, lnkf, 20)

    ci = Ncm.Vector.new(8)

    k_a = []
    Ps_zeta1 = []
    Ps_S1 = []
    Ps_zeta2 = []
    Ps_S2 = []
    Ps_Pzeta1 = []
    Ps_PS1 = []

    out_file = open("twofluids_spectrum_{w}.dat", "w", encoding="utf-8")
#alpha is related to the time variable. The log-redshift time
    start_alpha1 = 1.0e-10 # P5 tempo inicial dependente do redshift.
    start_alpha2 = 1.0e-14

    for lnk in tqdm(lnk_a):
        k = math.exp(lnk)
        pert.set_mode_k(k)
        k_a.append(k)

        alphaf = cosmo.abs_alpha(1.0e20)

        # print ("# Evolving mode %e from %f to %f" % (k, alphai, alphaf))
        #S is our variable Q.
        alphai = -cosmo.abs_alpha(start_alpha1 * k**2)

        #### Ate aqui só coloca todos os parametros, define o tempo inicial, e converte o alpha1 em algum tipo de numero absoluto nessa funçao de cima.

        ##### Agora embaixo, calcula as cond iniciais, seta as condicioes iniciais e evolui no tempo.

        pert.get_init_cond_zetaS(cosmo, alphai, 1, 0.25 * math.pi, ci)
#This function does the following: Calls get_init_cond_zetaS, which calls cond_QP.
#IN cond_QP:
        #Starts interface that have the equations of motion and the decomposition TV.
        #Defines which case: 1 or 2, defining which momentum we assume as non-zero
        #Computes all the init cond
        #I believe that R and I represents each solutioon 1 and 2.
#Then we call to_zetaS, which changes from QP to zeta Q
        pert.set_init_cond(cosmo, alphai, 1, False, ci)

        print(f"# Mode 1 k {k: 21.15e}, state module {pert.get_state_mod():f}")

        pert.evolve(cosmo, alphaf)#Evolve the system untill alphaF



        v, _alphac = pert.peek_state(cosmo)#Get the current time and values of the numerical solution for the modes. V é o vetor com os modos, n sei a ordem
#I believe that Delta represents the spectrum here.


##Aqui terminou de calcular os modos pro momento do loop. Ai agora calcula o espectro
        Delta_zeta1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.ZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.ZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_S1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.S_R), v.get(Nc.HIPertITwoFluidsVars.S_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_Pzeta1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.PZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_PS1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PS_R), v.get(Nc.HIPertITwoFluidsVars.PS_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )

        Ps_zeta1.append(Delta_zeta1)
        Ps_S1.append(Delta_S1)
        Ps_Pzeta1.append(Delta_Pzeta1)
        Ps_PS1.append(Delta_PS1)


        #### Aqui acabou o espectro pro modo 1. Ai faz de novo pro modo 2. ####3

        alphai = -cosmo.abs_alpha(start_alpha2 * k**2)
        pert.get_init_cond_zetaS(cosmo, alphai, 2, 0.25 * math.pi, ci)
        pert.set_init_cond(cosmo, alphai, 0, False, ci)

        pert.evolve(cosmo, alphaf)
        v, _alphac = pert.peek_state(cosmo)
        print(_alphac)
        Delta_zeta2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.ZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.ZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_S2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.S_R), v.get(Nc.HIPertITwoFluidsVars.S_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_Pzeta2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.PZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_PS2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PS_R), v.get(Nc.HIPertITwoFluidsVars.PS_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )

        Ps_zeta2.append(Delta_zeta2)
        Ps_S2.append(Delta_S2)
        Ps_zeta2.append(Delta_Pzeta2)
        Ps_S2.append(Delta_PS2)

        out_file.write(
            f"{k: 20.15e} {Delta_zeta1: 20.15e} {Delta_zeta2: 20.15e} {Delta_S1: 20.15e} "
            f"{Delta_S2: 20.15e} {Delta_Pzeta1: 20.15e} {Delta_Pzeta2: 20.15e} "
            f"{Delta_PS1: 20.15e} {Delta_PS2: 20.15e}\n"
        )
        out_file.flush()

    out_file.close()

###Plot

    plt.plot(k_a, Ps_zeta1, label=r"$P_\zeta$")
    plt.plot(k_a, Ps_S1, label=r"$P_Q$")
    #plt.plot(k_a, Ps_zeta2, label=r"$P^2_\zeta$")
   # plt.plot(k_a, Ps_S2, label=r"$P^2_Q$")
    plt.xlabel(r"$k $")
    plt.ylabel(r"$Spectrum$")
    plt.grid()
    plt.legend(loc="upper left")
    #plt.xscale("log")
    #plt.yscale("log")


    plt.savefig('spec')
    plt.show()

In [ ]:
def test_two_fluids_wkb_spec_log() -> None:
    """Compute WKB approximation for the two-fluids model spectrum."""

    #
    #  New homogeneous and isotropic cosmological model NcHICosmoQGRW
    #
    cosmo = Nc.HICosmoQGRW() # P1 = Cosmologia

    w = 0.00001 #P2 eq de estado da materia escura
    prec = 1.0e-7

    #P3 densidade de radiação e materia escura e energia escura (1 - resto)
    cosmo.props.w = w
    cosmo.props.Omegar = (1.0e-8) * 1.0
    cosmo.props.Omegaw = (1.0 - 1.0e-8) * 1.0
    cosmo.props.xb = 1.0e30 # P4 x do bounce

    pert = Nc.HIPertTwoFluids.new()

    pert.props.reltol = prec
    # pert.set_stiff_solver (True)

    lnki = math.log(1.0e-5)# P5 intervalo de momento de interesse
    lnkf = math.log(1.0e5)
    lnk_a = np.linspace(lnki, lnkf, 50)

    ci = Ncm.Vector.new(8)

    k_a = []
    Ps_zeta1 = []
    Ps_S1 = []
    Ps_zeta2 = []
    Ps_S2 = []
    Ps_Pzeta1 = []
    Ps_PS1 = []

    out_file = open("twofluids_spectrum_{w}.dat", "w", encoding="utf-8")
#alpha is related to the time variable. The log-redshift time
    start_alpha1 = 1.0e-10 # P5 tempo inicial dependente do redshift.
    start_alpha2 = 1.0e-14

    for lnk in tqdm(lnk_a):
        k = math.exp(lnk)
        pert.set_mode_k(k)
        k_a.append(k)

        alphaf = cosmo.abs_alpha(1.0e20)

        # print ("# Evolving mode %e from %f to %f" % (k, alphai, alphaf))
        #S is our variable Q.
        alphai = -cosmo.abs_alpha(start_alpha1 * k**2)

        #### Ate aqui só coloca todos os parametros, define o tempo inicial, e converte o alpha1 em algum tipo de numero absoluto nessa funçao de cima.

        ##### Agora embaixo, calcula as cond iniciais, seta as condicioes iniciais e evolui no tempo.

        pert.get_init_cond_zetaS(cosmo, alphai, 1, 0.25 * math.pi, ci)
#This function does the following: Calls get_init_cond_zetaS, which calls cond_QP.
#IN cond_QP:
        #Starts interface that have the equations of motion and the decomposition TV.
        #Defines which case: 1 or 2, defining which momentum we assume as non-zero
        #Computes all the init cond
        #I believe that R and I represents each solutioon 1 and 2.
#Then we call to_zetaS, which changes from QP to zeta Q
        pert.set_init_cond(cosmo, alphai, 1, False, ci)

        print(f"# Mode 1 k {k: 21.15e}, state module {pert.get_state_mod():f}")

        pert.evolve(cosmo, alphaf)#Evolve the system untill alphaF



        v, _alphac = pert.peek_state(cosmo)#Get the current time and values of the numerical solution for the modes. V é o vetor com os modos, n sei a ordem
#I believe that Delta represents the spectrum here.


##Aqui terminou de calcular os modos pro momento do loop. Ai agora calcula o espectro
        Delta_zeta1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.ZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.ZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_S1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.S_R), v.get(Nc.HIPertITwoFluidsVars.S_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_Pzeta1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.PZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_PS1 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PS_R), v.get(Nc.HIPertITwoFluidsVars.PS_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )

        Ps_zeta1.append(Delta_zeta1)
        Ps_S1.append(Delta_S1)
        Ps_Pzeta1.append(Delta_Pzeta1)
        Ps_PS1.append(Delta_PS1)


        #### Aqui acabou o espectro pro modo 1. Ai faz de novo pro modo 2. ####3

        alphai = -cosmo.abs_alpha(start_alpha2 * k**2)
        pert.get_init_cond_zetaS(cosmo, alphai, 2, 0.25 * math.pi, ci)
        pert.set_init_cond(cosmo, alphai, 0, False, ci)

        pert.evolve(cosmo, alphaf)
        v, _alphac = pert.peek_state(cosmo)
        print(_alphac)
        Delta_zeta2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.ZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.ZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_S2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.S_R), v.get(Nc.HIPertITwoFluidsVars.S_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_Pzeta2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PZETA_R),
                v.get(Nc.HIPertITwoFluidsVars.PZETA_I),
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )
        Delta_PS2 = (
            k**3
            * math.hypot(
                v.get(Nc.HIPertITwoFluidsVars.PS_R), v.get(Nc.HIPertITwoFluidsVars.PS_I)
            )
            ** 2
            / (2.0 * math.pi**2 * cosmo.RH_planck() ** 2)
        )

        Ps_zeta2.append(Delta_zeta2)
        Ps_S2.append(Delta_S2)
        Ps_zeta2.append(Delta_Pzeta2)
        Ps_S2.append(Delta_PS2)

        out_file.write(
            f"{k: 20.15e} {Delta_zeta1: 20.15e} {Delta_zeta2: 20.15e} {Delta_S1: 20.15e} "
            f"{Delta_S2: 20.15e} {Delta_Pzeta1: 20.15e} {Delta_Pzeta2: 20.15e} "
            f"{Delta_PS1: 20.15e} {Delta_PS2: 20.15e}\n"
        )
        out_file.flush()

    out_file.close()

###Plot

    plt.plot(np.log(k_a), np.log(Ps_zeta1), label=r"$P_\zeta$")
    plt.plot(np.log(k_a), np.log(Ps_S1), label=r"$P_Q$")
    #plt.plot(k_a, Ps_zeta2, label=r"$P^2_\zeta$")
   # plt.plot(k_a, Ps_S2, label=r"$P^2_Q$")
    plt.xlabel(r"$k $")
    plt.ylabel(r"$Spectrum$")
    plt.grid()
    plt.legend(loc="upper left")
    #plt.xscale("log")
    #plt.yscale("log")


    #plt.savefig('spec')
    plt.show()
    return np.log(Ps_zeta1), k_a

In [ ]:
#test_two_fluids_wkb_mode(Nc.HIPertTwoFluidsCross.MODE1MAIN, 1, 0.0, 1.0)
test_two_fluids_wkb_mode(Nc.HIPertTwoFluidsCross.MODE1MAIN, 1, 1.0, 0.0)
#test_two_fluids_wkb_mode(Nc.HIPertTwoFluidsCross.MODE1MAIN, 1, 1.0, 1.0)


In [ ]:
test_two_fluids_wkb_spec()

In [ ]:
test_two_fluids_wkb_spec_log()

In [ ]:
z1 = [-172.29336504, -171.47725159, -170.67740675, -169.40963547,
        -168.3753366 , -167.40418261, -166.617099  , -165.69081201,
        -164.60277739, -163.8977755 , -162.91261604, -161.89532066,
        -160.95146111, -159.82933117, -158.99936059, -158.1648807 ,
        -157.3978438 , -156.18665031, -155.4043561 , -154.44173584,
        -153.5056154 , -152.43139744, -151.48454113, -150.74967245,
        -149.71138271, -148.85570926, -147.97343268, -146.87265029,
        -145.89153898, -145.029969  , -143.98543597, -143.28854835,
        -142.18761254, -141.23608627, -140.32638846, -139.48824811,
        -138.47120016, -137.4639825 , -136.53337746, -135.66450856,
        -134.71280034, -133.5931724 , -132.540295  , -131.19474152,
        -129.90695834, -128.57067396, -127.17615592, -125.87341712,
        -124.6418451 , -123.48007829]

lnki = math.log(1.0e-5)# P5 intervalo de momento de interesse
lnkf = math.log(1.0e5)
lnk_a = np.linspace(lnki, lnkf, 50)
result=np.polyfit(lnk_a[:30], z1[:30], 1)

In [ ]:
print(result)

In [ ]:
ns = result[0] + 1
print(ns)

In [ ]:
12*1/3/(1+3*1/3) + 1